In [1]:
import numpy as np # linear algebra
import pandas as pd #
import glob
from tensorflow.keras.preprocessing.image import load_img, img_to_array


In [2]:

img_size = (224, 224)
img_array_list = []
cls_list = []

train_dir = 'input/1056lab-defect-detection/train/class'
for i in range(6):
    x = str(i + 1)
    img_list1 = glob.glob(train_dir + x + '/*.png')
    for i in img_list1:
        img = load_img(i, color_mode='grayscale', target_size=(img_size))
        img_array = img_to_array(img) / 255
        img_array_list.append(img_array)
        cls_list.append(0)

    img_list1 = glob.glob(train_dir + x + '_def/*.png')
    for i in img_list1:
        img = load_img(i, color_mode='grayscale', target_size=(img_size))
        img_array = img_to_array(img) / 255
        img_array_list.append(img_array)
        cls_list.append(1)

X_train = np.array(img_array_list)
y_train = np.array(cls_list)

In [1]:
X_train.shape

NameError: name 'X_train' is not defined

In [4]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
import tensorflow.keras.optimizers as opt

model = Sequential()
model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu', input_shape=(img_size[0], img_size[1], 1)))
model.add(MaxPooling2D(pool_size=(8, 8)))
model.add(Dropout(rate=0.5))

model.add(Conv2D(filters=128, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(4, 4)))
model.add(Dropout(rate=0.5))
model.add(Flatten())
model.add(Dense(units=1024, activation='relu'))

model.add(Dense(units=1, activation='sigmoid'))
Nadam = opt.Nadam(lr=8e-4, beta_1=0.9, beta_2=0.999, epsilon=None, schedule_decay=0.004)
model.compile(optimizer=Nadam, loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 222, 222, 128)     1280      
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 27, 27, 128)       0         
_________________________________________________________________
dropout (Dropout)            (None, 27, 27, 128)       0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 25, 25, 128)       147584    
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 128)         0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 6, 6, 128)         0         
_________________________________________________________________
flatten (Flatten)            (None, 4608)              0

In [5]:
from keras.utils.np_utils import to_categorical
model.fit(X_train, y_train, epochs=10, batch_size=64)

Epoch 1/10
108/108 [==============================] - 478s 4s/step - loss: 0.4129 - accuracy: 0.8661
Epoch 2/10
108/108 [==============================] - 474s 4s/step - loss: 0.3975 - accuracy: 0.8696
Epoch 3/10
108/108 [==============================] - 598s 6s/step - loss: 0.3918 - accuracy: 0.8696
Epoch 4/10
108/108 [==============================] - 481s 4s/step - loss: 0.3925 - accuracy: 0.8696
Epoch 5/10
108/108 [==============================] - 463s 4s/step - loss: 0.3891 - accuracy: 0.8696
Epoch 6/10
108/108 [==============================] - 463s 4s/step - loss: 0.3844 - accuracy: 0.8707
Epoch 7/10
108/108 [==============================] - 506s 5s/step - loss: 0.3472 - accuracy: 0.8867
Epoch 8/10
108/108 [==============================] - 485s 4s/step - loss: 0.3356 - accuracy: 0.8899
Epoch 9/10
108/108 [==============================] - 493s 5s/step - loss: 0.3315 - accuracy: 0.8900
Epoch 10/10
108/108 [==============================] - 541s 5s/step - loss: 0.3295 - accura

In [11]:
import glob
from keras.preprocessing.image import load_img, img_to_array

img_array_list = []

img_list = glob.glob('input/1056lab-defect-detection/test/*.png')
img_list.sort()
for i in img_list:
    img = load_img(i, color_mode='grayscale', target_size=(img_size))
    img_array = img_to_array(img) / 255
    img_array_list.append(img_array)

X_test = np.array(img_array_list)

In [23]:
predict = model.predict(X_test)[:, 0]
predict

array([0.18331897, 0.18005338, 0.18076473, 0.16115507, 0.16095033,
       0.16274872, 0.18381467, 0.16337696, 0.18160783, 0.18603288],
      dtype=float32)

In [21]:

submit = pd.read_csv('input/1056lab-defect-detection/submission.csv')
submit['defect'] = predict
submit.to_csv('submission.csv', index=False)